In [1]:
#Read in Redlist

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
style.use('ggplot')

# Read in csv as dataframe
rl = em.read_csv_metadata("../finalRedlist.csv", encoding="ISO-8859-1", key='id')
# glance at first few rows

ar = em.read_csv_metadata("../finalArkives.csv", encoding="ISO-8859-1", key='id')


c:\users\aparn\anaconda\envs\my_first_env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [2]:
rl['block'] = 'yes'
ar['block'] = 'yes'

In [3]:

rl.rename(columns={'Unnamed: 0':'Unnamed'}, inplace=True)
ar.rename(columns={'Unnamed: 0':'Unnamed'}, inplace=True)

In [13]:
columns = ar.columns

newColumns = []
for column in columns:
    column = column.lstrip().rstrip()
    newColumns.append(column)
    
ar.columns = newColumns

In [5]:
#Block on bogus attribute so that we can actually do
#blocking on genus
ob = em.OverlapBlocker()
C0 = ob.block_tables(rl, ar,'block', 'block', 
                     l_output_attrs=list(rl.columns.values), 
                     r_output_attrs=list(ar.columns.values))
len(C0)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:06


12687745

In [7]:
C1 = ob.block_candset(C0, 'genus', 'genus')
len(C1)

AssertionError: Right block attribute is not in the right table

In [7]:
feat = em.get_features_for_matching(rl, ar)

In [1]:
H = em.extract_feature_vecs(C1, 
                            feature_table=feat, 
                            attrs_after='rtable_block',
                            show_progress=False) 

NameError: name 'em' is not defined

In [26]:

#C1 = C1.drop('rtable_block', 1)
#C1 = C1.drop('ltable_block', 1)
result = pd.concat([C1, H], axis=1)
#result = result.drop('rtable_block',1)


ltable_Unnamed
ltable_block
rtable_Unnamed
rtable_block
Unnamed_Unnamed_exm
Unnamed_Unnamed_anm
Unnamed_Unnamed_lev_dist
Unnamed_Unnamed_lev_sim
block_block_lev_dist
block_block_lev_sim
block_block_jar
block_block_jwn
block_block_exm
block_block_jac_qgm_3_qgm_3
rtable_block


In [34]:
cols_to_drop = ['family_', 
                'scientific_name_', 
                'block', 
                'Unnamed',
                'name_name_cos_dlm_dc0_dlm_dc0',
                'name_name_mel', 
                'name_name_lev_dist',
                'name_name_lev_sim', 
                'name_name_nmw', 
                'name_name_sw'
               ]

for n in result.columns:
    for c in cols_to_drop:
        if c in n:
            result = result.drop(n,1)
result.columns

Index(['_id', 'ltable_id', 'rtable_id', 'ltable_name', 'ltable_genus',
       'ltable_ecology', 'ltable_countries', 'ltable_threat_paragraph',
       'ltable_conservation_paragraph', 'ltable_pop_trend', 'ltable_status',
       'ltable_country_count', 'rtable_name', 'rtable_kingdom',
       'rtable_phylum', 'rtable_class', 'rtable_order', 'rtable_genus',
       'rtable_size', 'rtable_threats', 'rtable_conservation',
       'rtable_threat_keywords', 'rtable_conservation_keywords',
       'rtable_tCount', 'rtable_cCount', '_id', 'ltable_id', 'rtable_id',
       'name_name_jac_qgm_3_qgm_3', 'name_name_cos_dlm_dc0_dlm_dc0',
       'name_name_jac_dlm_dc0_dlm_dc0', 'name_name_mel', 'name_name_lev_dist',
       'name_name_lev_sim', 'name_name_nmw', 'name_name_sw'],
      dtype='object')

In [220]:
label = em.split_train_test(C1, train_proportion=0.1039)
gold = label['train']
DS = label['test']


#Automatically label the training data
labels = []
matches = 0;
nonmatches = 0;
for index, row in gold.iterrows():
    if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
        labels.append(1)
        matches = matches + 1
    else:
        labels.append(0)
        nonmatches = nonmatches + 1
        
gold['label'] = labels
print('number of matches in training set:', matches)
print('number of nonmatches in training set:', nonmatches)

number of matches in training set: 50
number of nonmatches in training set: 350


In [221]:
gold


,_id,ltable_id,rtable_id,ltable_Unnamed,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,...,rtable_genus,rtable_size,rtable_threats,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_tCount,rtable_cCount,rtable_block,label
2111512,2111512,6122,266,6122,NaN,Shorea,Dipterocarpaceae,Terrestrial,"Brunei Darussalam; Indonesia (Sumatera); Malaysia (Peninsular Malaysia, Sarawak);",\r\r\n The population in Peninsular Malaysia has declined because of overexploitation.\r\r\n\...,...,Shorea,Trunk diameter: up to 172 cm (2),Dipterocarp forests have become amongst the most endangered in the world widely logged for use i...,Some subpopulations of this species are found in primary forest reserves where they receive vary...,,,0,0.0,yes,0
12415697,12415697,4977,1568,4977,hunaco water frog,Telmatobius,Telmatobiidae,Terrestrial; Freshwater,Peru;,\r\r\n Major threats include water pollution derived from agriculture and the use of agrochem...,...,Telmatobius,Length: 5 cm (2),<em>Telmatobius espadai </em>is now an extremely rare species; a population census undertaken in...,Despite <em>Telmatobius espadai </em>being classified as Critically Endangered by the IUCN Red L...,disease;,,1,0.0,yes,0
2111434,2111434,6044,266,6044,NaN,Shorea,Dipterocarpaceae,NaN,Malaysia (Peninsular Malaysia);,NaN,...,Shorea,Trunk diameter: up to 172 cm (2),Dipterocarp forests have become amongst the most endangered in the world widely logged for use i...,Some subpopulations of this species are found in primary forest reserves where they receive vary...,,,0,0.0,yes,0
2029104,2029104,2864,256,2864,NaN,Cycas,Cycadaceae,Terrestrial,Viet Nam;,\r\r\n Over-collection of plants from the wild has placed this plant at risk. It is a small s...,...,Cycas,Trunk height: to 7 m (2)Trunk diameter: 12 â?? 27 cm (2),While in many areas <i>Cycas circinalis</i> remains abundant in others its habitat has been seve...,Significant populations of <i>Cycas circinalis</i> still exist in a number of national parks and...,,,0,0.0,yes,0
2103797,2103797,6322,265,6322,NaN,Shorea,Dipterocarpaceae,Terrestrial,"Indonesia (Maluku, Sulawesi);",NaN,...,Shorea,Trunk diameter: up to 172 cm (2),Dipterocarp forests have become amongst the most endangered in the world widely logged for use i...,Some subpopulations of this species are found in primary forest reserves where they receive vary...,,,0,0.0,yes,0
7585347,7585347,2777,958,2777,"bornean agile gibbon, bornean whitebearded gibbon",Hylobates,Hylobatidae,Terrestrial,Indonesia (Kalimantan);,\r\r\n This species was certainly affected by fires of 1990s and continuing deforestation (le...,...,Hylobates,Head-body length: 45-50 cm (2)Male weight: c. 5.7 kg (3)Female weight: c.5.3 kg (3),Rapid loss of habitat poses the principle threat to gibbons placing their future in great peril ...,The white-handed gibbon is protected from international trade by its listing on Appendix I of th...,loss;pet;,cites;protected;,2,2.0,yes,0
7270109,7270109,4139,918,4139,NaN,Eleutherodactylus,Eleutherodactylidae,Terrestrial,Haiti;,"\r\r\n Severe habitat destruction is taking place in the Formon-Macaya region, including thro...",...,Eleutherodactylus,Female snout-vent length: up to 34 mm (2),Currently almost a quarter of all amphibian species on the British Virgin Islands are categorise...,A number of conservation recommendations have been made to protect the native fauna of the Briti...,,protected;,0,1.0,yes,0
11150720,11150720,6400,1408,6400,yellow meranti,Shorea,Dipterocarpaceae,Terrestrial,Singapore;,NaN,...,Shorea,Height: 70 m (2)Trunk diameter: 1.6 m (2),The Philippine mahogany has become depleted due to logging and <strong>slash-and-burn</strong> f...,Despite being Critically Endangered it is not clear whether any conservation measures exist for ...,,,0,0.0,yes,0
11218357,11218357,2802,1417,2802,NaN,Nepenthes,Nepenthaceae,Terrestrial,Malaysia (Sabah);,NaN,...,Nepenthes,Height: 3 m (2)Pitcher height: up to 

In [222]:
em.to_csv_metadata(gold, './gold.csv')
em.to_csv_metadata(DS, './unlabelled.csv')

File already exists at ./gold.csv; Overwriting it
Metadata file already exists at ./gold.metadata. Overwriting it
File already exists at ./unlabelled.csv; Overwriting it
Metadata file already exists at ./unlabelled.metadata. Overwriting it


True

In [223]:
test_train = em.split_train_test(gold, train_proportion=0.7)
train = test_train['train']
test = test_train['test']
len(train)

280

In [224]:
em.to_csv_metadata(gold, './train.csv')
em.to_csv_metadata(DS, './test.csv')

File already exists at ./train.csv; Overwriting it
Metadata file already exists at ./train.metadata. Overwriting it
File already exists at ./test.csv; Overwriting it
Metadata file already exists at ./test.metadata. Overwriting it


True

In [225]:
em.to_csv_metadata(result, './candidate_set.csv')


File already exists at ./candidate_set.csv; Overwriting it
Metadata file already exists at ./candidate_set.metadata. Overwriting it


True

In [226]:
result

,_id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,ltable_pop_trend,...,ltable_id,rtable_id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,name_name_nmw,name_name_sw
25215,25215,1470,3,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,NaN,Australia;,NaN,NaN,NaN,...,1470,3,0.738095,0.670820,0.500000,0.914987,18.0,0.632653,14.0,30.0
75598,75598,4363,9,albanian water frog,Pelophylax,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,"[""\n It is listed on Appendix III of the Bern Convention. 'Green frogs', including "", <em>R. ...",Decreasing,...,4363,9,0.791667,1.000000,1.000000,0.990000,1.0,0.950000,18.0,19.0
75600,75600,4365,9,NaN,Pelophylax,Terrestrial; Freshwater,China;,\r\r\n The major threat is over-collecting for food; other threats include drought and water ...,['\n The Shapotou location is within a national nature reserve. There is a need to manage the...,Decreasing,...,4365,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78778,78778,7543,9,karpathos frog,Pelophylax,Terrestrial; Freshwater,Greece;,\r\r\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,['\n It is listed on Appendix III of the Bern Convention. It probably occurs in a protected a...,Decreasing,...,7543,9,0.085714,0.408248,0.250000,0.573810,13.0,0.350000,1.0,6.0
83513,83513,4363,10,albanian water frog,Pelophylax,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,"[""\n It is listed on Appendix III of the Bern Convention. 'Green frogs', including "", <em>R. ...",Decreasing,...,4363,10,0.791667,1.000000,1.000000,0.990000,1.0,0.950000,18.0,19.0
83515,83515,4365,10,NaN,Pelophylax,Terrestrial; Freshwater,China;,\r\r\n The major threat is over-collecting for food; other threats include drought and water ...,['\n The Shapotou location is within a national nature reserve. There is a need to manage the...,Decreasing,...,4365,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86693,86693,7543,10,karpathos frog,Pelophylax,Terrestrial; Freshwater,Greece;,\r\r\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,['\n It is listed on Appendix III of the Bern Convention. It probably occurs in a protected a...,Decreasing,...,7543,10,0.085714,0.408248,0.250000,0.573810,13.0,0.350000,1.0,6.0
105744,105744,2849,13,"alexandria cycad, dune cycad",Encephalartos,Terrestrial,South Africa (Eastern Cape Province);,\r\r\n The major threat to this species is removal by collectors. This has been the main caus...,['\n This species is listed on Appendix I of the CITES Appendices. Populations occur within t...,Decreasing,...,2849,13,0.551724,0.707107,0.500000,0.921429,11.0,0.607143,6.0,16.0
105745,105745,2850,13,chimanimani cycad,Encephalartos,Terrestrial,Mozambique;,\r\r\n This species is threatened due to over-collecting for ornamental purposes.\r\r\n\r\r\n...,['\n This species is listed on Appendix I of the CITES Appendices.\n\n \n '],Decreasing,...,2850,13,0.129032,0.500000,0.333333,0.608824,10.0,0.411765,6.0,7.0
105746,105746,2851,13,NaN,Encephalartos,Terrestrial,"Tanzania, United Republic of;","\r\r\n This species is affected by too frequent fires, which may affect regeneration. Changes...",['\n This species is listed on Appendix I of the CITES Appendices. This species possibly occu...,Decreasing,...,2851,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
